<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 7: Correlations</h1>


<a name='section_7_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.0 Overview</h2>


<table style="width:100%">
    <colgroup>
       <col span="1" style="width: 40%;">
       <col span="1" style="width: 15%;">
       <col span="1" style="width: 45%;">
    </colgroup>
    <tr>
        <th style="text-align: left; font-size: 13pt;">Section</th>
        <th style="text-align: left; font-size: 13pt;">Exercises</th>
        <th style="text-align: left; font-size: 13pt;">Summary</th>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_1">L7.1 Understanding Best Fit (Revisited)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_1">L7.1 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_2">L7.2 Minimizing a Surface (1D Scan)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_2">L7.2 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_3">L7.3 Minimizing a Surface (2D Scan)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_3">L7.3 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_4">L7.4 Correlations Between Fit Parameters: Part 1</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_4">L7.4 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_5">L7.5 Correlations Between Fit Parameters: Part 2</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_5">L7.5 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
</table>



<h3>Learning Objectives</h3>

Text needed


<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook. 
Optionally, set the plot resolution and default figure size.


In [7]:
#>>>RUN

import numpy as np

#set plot resolution
#%config InlineBackend.figure_format = 'retina'

#set default figure size
#plt.rcParams['figure.figsize'] = (9,6)


<a name='section_7_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.1 Understanding Best Fit (Revisited)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_0) | [Exercises](#exercises_7_1) | [Next Section](#section_7_2) |


<h3>Slides</h3>

Run the code below to view the slides for this section.

In [5]:
#>>>RUN

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_01.html', width=970, height=550)

<h3>Uncertainty on more than one parameter</h3>

In the end of last week, we started to fit more than one parameter.  What does this mean? Well... this means we minimized two parameters instead of just one. If we have two parameters and they are both gaussian, then the sum of these two paramters is a $\chi^{2}$ of now **2** degrees of freedom. From this, we can start to understand what is going on. 

Let's go to our fit, and perform all the steps of the minimization that `lmfit` just did, but by hand. To do this we will again minimize the function 
\begin{equation}
 f(x,a,b) = a + b \sin(x)
\end{equation}
to the Auger data, which means that we need to find the mimimum of both $a$ and $b$, this means minimizing our likelihood for both $a$ and $b$. 

\begin{equation}
 \frac{\partial \mathcal{L}}{\partial a} = 0 \\
 \frac{\partial \mathcal{L}}{\partial b} = 0 \\
\end{equation}  

or writing this out using the $\chi^{2}$ we have, taking $\sigma = \sqrt{x_{i}}$

\begin{eqnarray}
\chi^{2}(x|a,b) &=& \sum_{i=1}^{N} \frac{(x_{i}-f(x_{i},a,b))^2}{x_{i}}\\
\frac{\partial \chi^{2}(x|a,b) }{\partial a} = 0 \\
\frac{\partial \chi^{2}(x|a,b) }{\partial b} = 0
\end{eqnarray}

The above minima is exactly what we need to do to minimize this. Lets go ahead and do that and look at how things vary. We will do this for each parameter treating them independently. 

In [ ]:
#>>>RUN

############ This is all code from lecture 4
import numpy as np
import csv
import math
import lmfit
import matplotlib.pyplot as plt
from scipy import optimize as opt

def rad(iTheta):
    return iTheta/180. * math.pi

def rad1(iTheta):
    return iTheta/180. * math.pi-math.pi

def load(label):
    dec=np.array([])
    ra=np.array([])
    az=np.array([])
    with open(label,'r') as csvfile:
        plots = csv.reader(csvfile,delimiter=' ')
        for pRow in plots:
            if '#' in pRow[0] or pRow[0]=='':
                continue
            dec = np.append(dec,rad(float(pRow[2])))
            ra  = np.append(ra,rad1(float(pRow[3])))
            az  = np.append(az,rad(float(pRow[4])))
    return dec,ra,az

def prephist(iRA):
    y0, bin_edges = np.histogram(iRA, bins=30)
    x0 = 0.5*(bin_edges[1:] + bin_edges[:-1])
    y0 = y0.astype('float')
    return x0,y0,1./(y0**0.5)

label8='events_a8_1space.dat'
dec,ra8,az=load(label8)
xhist,yhist,xweights=prephist(ra8)


########## Tlast fit code

def fnew(x,a,b):
    pVal=b*np.sin(x)
    return a+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=10)
result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()


In [ ]:
import scipy.stats as stats
print(stats.norm.cdf(1)-stats.norm.cdf(-1))
def pval(iVal):
    return stats.norm.cdf(iVal)-stats.norm.cdf(-iVal)

def chi2Val(iGausSigma,iNDOF):
    val=stats.chi2.ppf(pval(iGausSigma),iNDOF)
    return val
print(chi2Val(1,2))

<a name='exercises_7_1'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_1) | [Next Section](#section_7_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 7.1.1</span>

text


In [13]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_7_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.2 Minimizing a Surface (1D Scan)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_1) | [Exercises](#exercises_7_2) | [Next Section](#section_7_3) |


<h3>Overview</h3>

TEXT


In [ ]:
#This is the new code
def chi2(iX):
    lTot=0
    for val in range(len(yhist)):
        xtest=fnew(xhist[val],iX[0],iX[1])
        lTot = lTot+(1./(xtest+1e-5))*(yhist[val]-xtest)**2
    return lTot

def chi2minX(xval):
    val=chi2([xval,sol.x[1]])
    minval=chi2(sol.x)+1
    return val-minval

def chi2minY(yval):
    val=chi2([sol.x[0],yval])
    minval=chi2(sol.x)+1
    return val-minval

def chi2uncX(sol):
    solX1=opt.root_scalar(chi2minX,bracket=[sol.x[0], sol.x[0]*1.02],method='brentq')
    solX2=opt.root_scalar(chi2minX,bracket=[sol.x[0]*0.98, sol.x[0]],method='brentq')
    print("a:",sol.x[0],"+/-",abs(solX2.root-solX1.root)/2.)
    print("Reminder the Poission uncertianty would be:",math.sqrt(sol.x[0]/40))
    return solX1, solX2

def chi2uncY(sol):
    solY1=opt.root_scalar(chi2minY,bracket=[sol.x[1],    sol.x[1]*1.2],method='brentq')
    solY2=opt.root_scalar(chi2minY,bracket=[sol.x[1]*0.8, sol.x[1]],method='brentq')
    print("b:",sol.x[1],"+/-",abs(solY2.root-solY1.root)/2.)
    return solY1, solY2


#First we minimize
x0 = np.array([1000,10]) # initial conditions
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

In [ ]:
#Scan near the minimum of each value
x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.5,sol.x[1]*1.5, 100)

#Now lets fix one parameter at the minimum, and profile the other
plt.plot(x, chi2([x,sol.x[1]]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("b-value")
plt.ylabel("$\chi^{2}$")
plt.show()

#Now for the other parameter
plt.plot(y, chi2([sol.x[0],y]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("b-value")
plt.ylabel("$\chi^{2}$")
plt.show()

solX1, solX2 = chi2uncX(sol)
solY1, solY2 = chi2uncY(sol)

So now we have used our minimizer in 2D to profile the $\chi^{2}$ distribution, and we have obtained an uncertainty by looking at $\Delta \chi^{2}$, and you can see that our semi by-hand manipulation got us to the best parameters of the fitting. Additionally by profiling $\chi^{2}$ in each of the single parameters we obtained the same uncertiinties. 

As a small note our final fitted uncertainty is a little bit larger than the poission uncertainty. In fact for all fits there is a rule that our uncertainties on any parameter have to be larger than a certain number. This bound is known as the Cramér-Rao bound. I won't derive it or go into in detail, but the Cramér-Rao bound states. 
\begin{equation}
\mathrm{Var}(\theta|\hat{\theta}) \geq \frac{1}{\mathcal{I}(\theta)} \\
\mathcal{I}(\theta) = E_{p(X|\theta)}\left[-\frac{\partial^{2}}{\partial\theta^{2}}\log\left(p\left(x|\theta\right)\right)\right]
\end{equation}
Where here we call $\mathcal{I}(\theta)$ the Fisher information. While I don't want to go into this more, this result is powerful because it means that there is limit to when you should stop searching for a best fit. 

<a name='exercises_7_2'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_2) | [Next Section](#section_7_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 7.2.1</span>

text


In [23]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_7_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.3 Minimizing a Surface (2D Scan)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_2) | [Exercises](#exercises_7_3) | [Next Section](#section_7_4) |


<h3>Slides</h3>

Run the code below to view the slides for this section.

In [4]:
#>>>RUN

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_03.html', width=970, height=550)

To really visualize the whole thing lets make one more plot the 2D $\chi^2$ distribution. 

In [ ]:
#define the 2D X and Y grid
x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100) #grid in x
y = np.linspace(sol.x[1]*0.5,sol.x[1]*1.5, 100)#grid in y
X, Y = np.meshgrid(x, y) #2d grid

#Now our z-axis will be the chi2 of the 2D grid (this is complicated code)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)

#and plot
def plotColorsAndContours(X,Y,Z):
    fig, ax = plt.subplots(1, 1)
    c = ax.pcolor(X,Y,Z,cmap='RdBu')
    cb=fig.colorbar(c, ax=ax)
    plt.xlabel("a")
    plt.ylabel("b")
    cb.set_label("$\chi^{2}$")
    #Now lets plot the isobars of Delta chi^2
    levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
    for i0 in range(len(levels)):
        levels[i0] = levels[i0]+sol.fun
    c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
    #plt.show()
    
plotColorsAndContours(X,Y,Z)

Now lets got back to our fit, we obtained the 1D results, but we made this 2D plot, and we drew a funny yellow line on it above. What exactly is the uncertainty profile when we are profiling 2 variables at once. Lets go back to our our Taylor expansion result above, but lets write it in terms of all variables $\vec{\theta}$:  

\begin{equation}
\chi^{2}(x_{i},\vec{\theta})=\chi^{2}_{min}(x_{i},\vec{\theta})+\frac{1}{2}(\theta_{i}-\theta_{j})^{T}\frac{\partial^{2}}{\partial \theta_{i}\theta_{0}}\chi^{2}_{min}(x_{i},\vec{\theta}_{0})(\theta_{j}-\theta_{0})
\end{equation}
We can write this out in 2D as:
\begin{equation}
\chi^{2}(x,\vec{\theta})=\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\theta_{a}-\theta_{a-min} & \theta_{b}-\theta_{b-min}\end{array}\right)\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\\
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\left(\begin{array}{c}
\theta_{a}-\theta_{a-min}\\
\theta_{b}-\theta_{b-min}
\end{array}\right)
\end{equation}
Now in the case where $\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\approx0$ we can simplify this distribution by a lot. Lets do it 

\begin{equation}
\chi^{2}(x,\vec{\theta})=\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\Delta\theta_{a} & \Delta\theta_{b}\end{array}\right)\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & 0\\
0 & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\left(\begin{array}{c}
\Delta\theta_{a}\\
\Delta\theta_{b}
\end{array}\right)
\end{equation}

This all becomes a 2D quadratic equation

\begin{align*}
\chi^{2}(x,\vec{\theta}) & =\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\Delta\theta_{a}^{2}\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}}+ & \Delta\theta_{b}^{2}\frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}\end{array}\right)\\
 & =\chi_{min}^{2}(x,\vec{\theta})+\left(\begin{array}{cc}
\frac{\Delta\theta_{a}^{2}}{\sigma_{\theta_{a}}^{2}}+ & \frac{\Delta\theta_{b}^{2}}{\sigma_{\theta_{b}}^{2}}\end{array}\right)
\end{align*}

So let's plot this quadratic and compare it to our minimum contours. 

Now before we do this I want to point out one more thing, we are now profiling two parameters at once in this 2D plot, which means the sum of our independent variables will be a $\chi^{2}$ for 2 degrees of freedom. The 1 $\sigma$ confidence interval for 2-degrees of freedom is computed by taking $\Delta \chi^2(x,\nu=2)=x~\mathrm{where~}\mathrm{cdf}\left(\chi^{2}(x,2)=0.683\right)\approx2.3$. This is the yellow line on the contour.


In [ ]:
#Lets plot the uncertainties  from hess_inv
print(np.sqrt(2*sol.hess_inv))
#the diagonals are approximately the errors

#Make a the expression in the above equation x and x0 are 2 vectors
def quadratic2D(x,x0,sigma0):
    lVals=x-x0
    lVals=(lVals**2)/(sigma0)/sigma0
    return np.sum(lVals)

plotColorsAndContours(X,Y,Z)

#Now plot the quadratic circles
def plotQuadrticCricles(sigx,sigy):
    levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
    ZQ = np.array([quadratic2D([x,y],sol.x,[sigx,sigy]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
    c = plt.contour(X, Y, ZQ, levels,colors=['red', 'blue', 'yellow','green'],linestyles='dashed')

sigx=(solX2.root-solX1.root)/2.
sigy=(solY2.root-solY1.root)/2.
plotQuadrticCricles(sigx,sigy)
plt.show()

This gives us the same yellow line for our variance band. If you look close you do see a difference. This makes us beg the question of what happens when $\frac{\partial^{2}\chi^{2}}{\partial \theta_{a}\theta_{b}}\neq0$. Lets try another two parameter fit. To see that, lets try to minimize with a different function. 

<a name='exercises_7_3'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_3) | [Next Section](#section_7_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 7.3.1</span>

text


In [33]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_7_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.4 Correlations Between Fit Parameters: Part 1</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_3) | [Exercises](#exercises_7_4) | [Next Section](#section_7_5) |


<h3>Slides</h3>

Run the code below to view the slides for this section.

In [1]:
#>>>RUN

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_04.html', width=970, height=550)

### Challenge question:

Make the above 2D contour plot with circles using the minimization function

\begin{equation}
 f(x) = a x + b (1-x)
\end{equation}

Try to use the quadratic2D approximation, why does the 2D quadratic not agree? Why are the uncertainties so far off? 

In [ ]:
def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=1000)

result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()


x0 = np.array([1000,1000])
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
plotColorsAndContours(X,Y,Z)

solX1, solX2 = chi2uncX(sol)
solY1, solY2 = chi2uncY(sol)
plotQuadrticCricles(sigx,sigy)
plt.show()


<a name='exercises_7_4'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_4) | [Next Section](#section_7_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 7.4.1</span>

text


In [43]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_7_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.5 Correlations Between Fit Parameters: Part 2</h2>     

| [Top](#section_7_0) | [Previous Section](#section_7_4) | [Exercises](#exercises_7_5) |


<h3>Dealing with Correlated uncertainties</h3>

Now look at our parameters our uncertainty estimate is smaller than what we observed above. Incidently the uncertainties quoted now differ from what we got by varying from the $\chi^{2}$ minimum. What we are doing is moving up and down the 2D plot and looking at $\delta \chi^{2}$. However given the parameters are so correlated, you see that this isn't really a reflection of the true uncertainty in the sense that we can move along $x$ and $y$ and still be within the yellow or even blue ellipse. It's quite clear when you overlap the uncertainty from the quadratic function, which draws circles not ellipses. 

So then what is the right uncertainty? 

Well from our fit using lmfit above we see that the fit outputs a parameter $C(a,b)=0.872$ this is the correlation between the parameters of $a$ and $b$ we also see from our optimization function, we get a computation of something labelled as the hess_inv. This we can write noting the relation of the $\chi^{2}$ Hessian and uncertainties as 
\begin{equation}
\chi^{2}(x_{i},\vec{\theta})=\chi^{2}_{min}(x_{i},\vec{\theta})+\frac{1}{2}(\theta_{i}-\theta_{j})^{T}\frac{\partial^{2}}{\partial \theta_{i}\theta_{0}}\chi^{2}_{min}(x_{i},\vec{\theta}_{0})(\theta_{j}-\theta_{0})
\end{equation}

Which allows us to write the uncertainties as
\begin{equation}
 \frac{2}{\sigma^{2}}=\frac{\partial^{2}\chi^{2}}{\partial\theta_{i}\partial\theta_{j}} \\
 \sigma^{2}    = 2\left(\frac{\partial^{2}\chi^{2}}{\partial\theta_{i}\partial\theta_{j}}\right)^{-1} 
\end{equation}

Now in the case where the off diagonals of the Hessian were zero we had the clear case that the samples are not correlated, and as a consequence, 
\begin{equation}
\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & 0\\
0 & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\rightarrow\left(\begin{array}{cc}
\frac{2}{\sigma_{a}^{2}} & 0\\
0 & \frac{2}{\sigma_{b}^{2}}
\end{array}\right)
\end{equation}

Now we have something more complicated. However this is a natural way to define correlations. Lets first verify our intuition for our minimization scheme by taking our $2x2$ Hessian metrix and diagonalizing, computing the eigenvectors and the eigenvalues and then drawing an ellipse. We can diagonalize the matrix as: 
\begin{equation}
A^{-1}2\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\\
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)^{-1}A=\left(\begin{array}{cc}
\sigma_{1}^{2} & 0\\
0 & \sigma_{2}^{2}
\end{array}\right)
\end{equation}
It is important to note for any N dimensional hessian, provided the determinant is not zero, we can find an basis of independent variables that are not correlated. That is to say we can always diagonalize our Hessian, and the eignevectors of our Hessian are the independent values with variances given by the eigenvalues. 

Knowing how this works, lets play with the Hessian. When we run our minimizer, we get Hessian inverse, let's play with it. 


In [50]:
#>>>RUN

print(np.sqrt(2*sol.hess_inv))
#The diagonals are the uncertainty lmfit quotes

#Really the best way to do this is to get the eigen values using an linear algebra problem
import numpy.linalg as la
w, v=la.eig(2*sol.hess_inv)
print("values",w,"vectors",v)

#Now lets plot the eigenvectors
from matplotlib.patches import Ellipse
def get_cov_ellipse(cov, centre, nstd, **kwargs):
    """
    Return a matplotlib Ellipse patch representing the covariance matrix
    cov centred at centre and scaled by the factor nstd.

    """
    # Find and sort eigenvalues and eigenvectors into descending order
    eigvals, eigvecs = np.linalg.eigh(cov)
    order = eigvals.argsort()[::-1]
    eigvals, eigvecs = eigvals[order], eigvecs[:, order]

    # The anti-clockwise angle to rotate our ellipse by 
    vx, vy = eigvecs[:,0][0], eigvecs[:,0][1]
    theta = np.arctan2(vy, vx)

    # Width and height of ellipse to draw
    width, height = 2 * nstd * np.sqrt(eigvals)
    return Ellipse(xy=centre, width=width, height=height,angle=np.degrees(theta), **kwargs)

err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x,1)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
for i0 in range(len(levels)):
    levels[i0] = levels[i0]+sol.fun
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.show()

From the above we can now formulate how we should describe the uncertinaties in our system. When we had a decorrelated system we had a total uncrtainty in our $\chi^{2}$ given by 
\begin{equation}
\sigma_{tot}^{2} = \sigma_{a}^2+\sigma_{b}^2
\end{equation}

Now we have the full ellipse
\begin{equation}
\sigma_{tot}^{2} = \sigma_{a}^2+\sigma_{b}^2+2\sigma_{ab}
\end{equation}

Where we define that $\sigma_{ab}=\rm{COV(a,b)}$. There are a number of ways to call this variable, they all are equivalent, but lets be careful to write it out. We can write the error matrix in 2D as:
\begin{equation}
\left(\begin{array}{cc}
\sigma_{a}^{2} & {\rm COV}(a,b)\\
{\rm COV}(a,b) & \sigma_{b}^{2}
\end{array}\right)=\sum_{i=1}^{N}\left(\begin{array}{cc}
\left(a_{i}-\bar{a}\right)^{2} & \left(a_{i}-\bar{a}\right)\left(b_{i}-\bar{b}\right)\\
\left(a_{i}-\bar{a}\right)\left(b_{i}-\bar{b}\right) & \left(b_{i}-\bar{b}\right)^{2}
\end{array}\right)
\end{equation}
where on the right side we have written it in terms of the computation over events. Recall that for the a linear regression the slope is just the $\rm{COV(X,Y)/VAR(X)}$, so the covariance matrix is intricately tied with slope. 

We can also write it as the correlation matrix where we normalize by the uncertainties:
\begin{equation}
\rho=\left(\frac{1}{\sigma_{a}}  \frac{1}{\sigma_{b}} \right)^{T}\left(\begin{array}{cc}
1 & \frac{{\rm COV}(a,b)}{\sigma_{a}\sigma_{b}}\\
\frac{{\rm COV}(a,b)}{\sigma_{a}\sigma_{b}} & 1
\end{array}\right)\left(\frac{1}{\sigma_{a}}   \frac{1}{\sigma_{b}} \right) 
\end{equation}

Recall that the covariance is what we originally used to compute the linear slope of the points, this is exactly the same here. In fact, instead of scanning the likelihood analytically we could have sampled the points, and done a linear regression. The resulting slope and line can be related to our eigenvectors. One last think to mention is that if variables are correlated, we can use the correlation to propagate the uncertainties. 
\begin{equation}
\sigma_{f}^{2} = \left(\frac{\partial f}{\partial x}\right)^2\sigma_{x}^2 + \left(\frac{\partial f}{\partial y}\right)^{2}+\left(\frac{\partial f}{\partial x}\right)\left(\frac{\partial f}{\partial y}\right)\sigma_{xy}
\end{equation}

Now lets check we can get the corelation matrix.

In [50]:
#Now lets get the correlation C(a,b) (see below)
w, v=np.linalg.eig(2*sol.hess_inv)
print("c(a,b)",v[0,1]/v[0,0])
print("A deceptively wrong way to get correlation:",sol.hess_inv[0,1]/sol.hess_inv[0,0])

Finally, let's compute it from some toy data. 

In [50]:
import lmfit

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

#Randomly sample points in the above range
def maketoy(iy):
    toy=np.array([])
    for i0 in range(len(iy)):
        pVal = np.random.normal (iy[i0],np.sqrt([iy[i0]]))
        toy = np.append(toy,float(pVal))
    return toy

def fittoy(ibin,iy):
    toy=maketoy(iy)
    model  = lmfit.Model(fnew)
    p = model.make_params(a=1000,b=10)
    xweights=np.array([])
    for i0 in range(len(toy)):
        xweights = np.append(xweights,1./math.sqrt(toy[i0]))
    result = model.fit(data=toy,x=ibin, params=p, weights=xweights)
    return result.params["a"].value,result.params["b"].value

ntoys=1000
lAs=np.array([])
lBs=np.array([])
for i0 in range(ntoys):
    pA,pB=fittoy(xhist,yhist)
    lAs = np.append(lAs,pA)
    lBs = np.append(lBs,pB)

err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x, 1)

fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.plot(lAs,lBs,c='black',marker='.',linestyle = 'None')
plt.show()

#Now lets run a. linear regression
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

print("A:",lAs.mean(),"+/-",lAs.std())
print("B:",lBs.mean(),"+/-",lBs.std())
print("Cov:",covariance(lAs,lBs),"A Variance:",variance(lAs),"B Variance:",variance(lBs))
print("Check with Hessian:",2*sol.hess_inv)
print("Cor:",covariance(lAs,lBs)/math.sqrt(variance(lAs)*variance(lBs)),"A Variance:",1.,"B Variance:",1.)

<a name='exercises_7_5'></a>   

| [Top](#section_7_0) | [Restart Section](#section_7_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 7.5.1</span>

text


In [53]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>
